In [3]:
pip install eli5

     |████████████████████████████████| 112kB 5.0MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 
os.chdir(r"/content/drive/MyDrive/SDA/Dataset") 

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as mno
from sklearn import linear_model
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer 
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model

In [5]:
data = pd.read_csv("Whoop-Data.csv")
data = data.drop(["Sleep Onset","Sleep Onset","Wake Onset","Last Nap End Time","timezone_offset","Nap Time in Bed","Nap Disturbances","Nap REM","Nap SWS","Nap Light","Nap Wake","Nap Missing Data"],axis=1)
data.drop(data.iloc[:, 0:4], inplace = True, axis = 1) 
data

,RHR,HRV,Recovery,Sleep Score,Hours in Bed,Hours of Sleep,Sleep Need,Sleep Efficiency (%),Wake Periods,Sleep Disturbances,Latency (min),Cycles,REM Sleep (hours),Deep Sleep (hours),Light Sleep (hours),Awake (hours),Missing Data (hours),Sleep Debt (hours),Sleep Consistency,Respiratory Rate,Total Cycle Nap Time (hours),Total Cycle Sleep Time (hours),REM Percentage,Deep Sleep Percentage,Restorative Sleep (hours),Restorative Sleep (%)
0,51,68,43,76,7.71,7.18,9.47,93,8.0,7,1.51,4,1.74,1.60,3.84,0.53,0.0,1.61,76.0,15.0000,0.00,7.18,22.6,20.8,3.34,43.3
1,54,80,65,62,6.66,6.04,9.75,91,10.0,8,4.77,5,2.22,1.00,2.82,0.62,0.0,1.19,74.0,16.1719,0.00,6.04,33.3,15.0,3.22,48.3
2,55,74,58,80,8.84,8.10,10.07,92,12.0,8,3.33,8,2.94,0.92,4.24,0.75,0.0,1.46,74.0,16.1133,0.00,8.10,33.3,10.4,3.86,43.7
3,49,59,43,77,7.76,7.40,9.60,95,8.0,7,1.87,3,2.45,1.80,3.15,0.37,0.0,0.99,62.0,15.3223,0.00,7.40,31.6,23.2,4.25,54.8
4,53,29,15,78,8.69,7.43,9.51,86,15.0,9,8.16,7,2.19,1.98,3.27,1.27,0.0,1.14,72.0,15.4980,0.00,7.43,25.2,22.8,4.17,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1805,64,52,36,78,8.11,7.35,9.37,91,10.0,7,2.11,6,3.09,1.27,3.00,0.77,0.0,1.46,NaN,15.1758,0.00,7.35,38.1,15.7,4.36,53.8
1806,57,68,56,86,8.40,7.41,8.62,88,14.0,13,3.04,6,2.85,1.67,2.90,1.00,0.0,0.85,79.0,15.0000,0.00,7.41,33.9,19.9,4.52,53.8
1807,56,83,76,100,9.93,8.87,8.48,89,14.0,11,2.48,8,3.41,2.43,3.04,1.07,0.0,0.69,76.0,14.8828,0.00,8.87,34.3,24.5,5.84,58.8
1808,52,77,68,91,7.76,7.03,7.77,91,11.0,10,3.17,5,2.91,1.28,2.84,0.74,0.0,0.00,74.0,15.0586,0.00,7.03,37.5,16.5,4.19,54.0


In [6]:
data.describe()

,RHR,HRV,Recovery,Sleep Score,Hours in Bed,Hours of Sleep,Sleep Need,Sleep Efficiency (%),Wake Periods,Sleep Disturbances,Latency (min),Cycles,REM Sleep (hours),Deep Sleep (hours),Light Sleep (hours),Awake (hours),Missing Data (hours),Sleep Debt (hours),Sleep Consistency,Respiratory Rate,Total Cycle Nap Time (hours),Total Cycle Sleep Time (hours),REM Percentage,Deep Sleep Percentage,Restorative Sleep (hours),Restorative Sleep (%)
count,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1486.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1810.000000,1599.000000,1807.000000,1810.000000,1739.000000,1739.000000,1739.000000,1739.000000,1739.000000
mean,58.647514,84.658011,58.159669,76.338122,7.565735,6.806580,8.872210,90.183978,12.712651,9.735912,1.861691,5.032044,2.164061,1.371696,3.270713,0.763376,0.018663,0.995061,64.652283,16.721831,0.292602,7.085170,28.581599,18.109316,3.530253,46.691604
std,8.388280,39.474389,22.729122,17.747773,1.734912,1.522441,1.071613,5.534741,6.051647,5.032002,3.709872,1.926244,0.822354,0.436119,1.049201,0.521806,0.360070,0.708367,14.232265,1.336336,0.803994,1.652522,9.137148,4.564252,1.095139,10.754627
min,43.000000,11.000000,1.000000,12.000000,1.260000,1.070000,3.350000,55.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.480000,0.000000,0.000000,0.000000,3.000000,13.418000,0.000000,1.070000,0.000000,0.000000,0.000000,0.000000
25%,53.000000,56.000000,41.000000,64.000000,6.540000,5.910000,8.250000,87.000000,8.000000,6.000000,0.000000,4.000000,1.590000,1.090000,2.590000,0.410000,0.000000,0.440000,55.500000,15.644500,0.000000,6.125000,22.300000,15.400000,2.810000,40.000000
50%,57.000000,80.000000,59.000000,77.000000,7.610000,6.900000,8.910000,91.000000,12.000000,9.000000,1.760000,5.000000,2.170000,1.390000,3.190000,0.630000,0.000000,0.990000,66.000000,16.875000,0.000000,7.100000,29.300000,18.300000,3.560000,47.600000
75%,63.000000,107.000000,76.000000,91.000000,8.650000,7.780000,9.577500,94.000000,16.000000,12.750000,2.950000,6.000000,2.750000,1.670000,3.880000,0.950000,0.000000,1.480000,75.000000,17.578100,0.000000,8.065000,35.000000,21.150000,4.250000,54.200000
max,89.000000,224.000000,99.000000,100.000000,20.640000,19.270000,11.790000,100.000000,38.000000,31.000000,138.220000,12.000000,4.980000,2.740000,16.520000,5.530000,12.230000,3.400000,93.000000,20.097700,8.010000,19.270000,56.100000,32.200000,6.920000,81.500000


In [7]:
X = data.iloc[:,1:len(data.columns)-2]
data.head()


,RHR,HRV,Recovery,Sleep Score,Hours in Bed,Hours of Sleep,Sleep Need,Sleep Efficiency (%),Wake Periods,Sleep Disturbances,Latency (min),Cycles,REM Sleep (hours),Deep Sleep (hours),Light Sleep (hours),Awake (hours),Missing Data (hours),Sleep Debt (hours),Sleep Consistency,Respiratory Rate,Total Cycle Nap Time (hours),Total Cycle Sleep Time (hours),REM Percentage,Deep Sleep Percentage,Restorative Sleep (hours),Restorative Sleep (%)
0,51,68,43,76,7.71,7.18,9.47,93,8.0,7,1.51,4,1.74,1.60,3.84,0.53,0.0,1.61,76.0,15.0000,0.0,7.18,22.6,20.8,3.34,43.3
1,54,80,65,62,6.66,6.04,9.75,91,10.0,8,4.77,5,2.22,1.00,2.82,0.62,0.0,1.19,74.0,16.1719,0.0,6.04,33.3,15.0,3.22,48.3
2,55,74,58,80,8.84,8.10,10.07,92,12.0,8,3.33,8,2.94,0.92,4.24,0.75,0.0,1.46,74.0,16.1133,0.0,8.10,33.3,10.4,3.86,43.7
3,49,59,43,77,7.76,7.40,9.60,95,8.0,7,1.87,3,2.45,1.80,3.15,0.37,0.0,0.99,62.0,15.3223,0.0,7.40,31.6,23.2,4.25,54.8
4,53,29,15,78,8.69,7.43,9.51,86,15.0,9,8.16,7,2.19,1.98,3.27,1.27,0.0,1.14,72.0,15.4980,0.0,7.43,25.2,22.8,4.17,48.0


In [8]:
missing_columns=["Wake Periods","Sleep Consistency","Respiratory Rate", "Total Cycle Sleep Time (hours)","REM Percentage", "Deep Sleep Percentage", "Restorative Sleep (hours)", "Restorative Sleep (%)"]


In [9]:
def random_imputation(df, feature):

    number_missing = df[feature].isnull().sum()
    observed_values = df.loc[df[feature].notnull(), feature]
    df.loc[df[feature].isnull(), feature + '_imp'] = np.random.choice(observed_values, number_missing, replace = True)
    
    return df

In [11]:
for feature in missing_columns:
    data[feature + '_imp'] = data[feature]
    data = random_imputation(data, feature)

In [18]:
deter_data = pd.DataFrame(columns = ["Det" + name for name in missing_columns])

for feature in missing_columns:
        
    deter_data["Det" + feature] = data[feature + "_imp"]
    A=data.copy()
    parameters = list(set(data.columns) - set(missing_columns) - {feature + '_imp'})
 
    imputer = SimpleImputer(missing_values=np.nan,strategy="mean")
    imputer.fit(A.iloc[:,0:])
    A.iloc[:,0:] = imputer.transform(A.iloc[:,0:])
    
    #Create a Linear Regression model to estimate the missing data
    model = linear_model.LinearRegression()
    model.fit(X = A[parameters], y = data[feature + '_imp'])
    
    #observe that I preserve the index of the missing data from the original dataframe
    deter_data.loc[data[feature].isnull(), "Det" + feature] = model.predict(A[parameters])[data[feature].isnull()]
    
    #predict=model.predict(A[parameters])
    #std_error = (predict[data[feature].notnull()] - data.loc[data[feature].notnull(), feature + '_imp']).std()
    
    #print(std_error)

print(deter_data)
pd.concat([data["Wake Periods"],deter_data["DetWake Periods"]], axis = 1).describe().T
#pd.concat([data["Sleep Consistency"],deter_data["DetSleep Consistency"]], axis = 1).describe().T
Y=pd.DataFrame(deter_data)
Y.to_csv('/content/drive/MyDrive/SDA/Dataset/LR_modified_columns_final.csv')


      DetWake Periods  ...  DetRestorative Sleep (%)
0                 8.0  ...                      43.3
1                10.0  ...                      48.3
2                12.0  ...                      43.7
3                 8.0  ...                      54.8
4                15.0  ...                      48.0
...               ...  ...                       ...
1805             10.0  ...                      53.8
1806             14.0  ...                      53.8
1807             14.0  ...                      58.8
1808             11.0  ...                      54.0
1809             20.0  ...                      46.8

[1810 rows x 8 columns]


In [19]:
random_data = pd.DataFrame(columns = ["Ran" + name for name in missing_columns])

for feature in missing_columns:
        
    random_data["Ran" + feature] = data[feature + "_imp"]
    B=data.copy()
    parameters = list(set(data.columns) - set(missing_columns) - {feature + '_imp'})
 
    imputer = SimpleImputer(missing_values=np.nan,strategy="mean")
    imputer.fit(B.iloc[:,0:])
    B.iloc[:,0:] = imputer.transform(B.iloc[:,0:])
    
#Create a Linear Regression model to estimate the missing data
    model = linear_model.LinearRegression()
    model.fit(X = B[parameters], y = data[feature + '_imp'])
    
    predict=model.predict(B[parameters])
    std_error = (predict[B[feature].notnull()] - B.loc[B[feature].notnull(), feature + '_imp']).std()
    random_predict = np.random.normal(size = A[feature].shape[0], loc = predict, scale = std_error)
    random_data.loc[(A[feature].isnull()) & (random_predict > 0), "Ran" + feature] = random_predict[(A[feature].isnull()) & (random_predict > 0)]

random_data
Z=pd.DataFrame(deter_data)
Z.to_csv('/content/drive/MyDrive/SDA/Dataset/SLR_modified_columns_final.csv')

Trying LR and SLR with RHR attribute (removing values randomly)

In [20]:
data_RHR = pd.read_csv("Whoop-Data.csv")
data_RHR = data_RHR.drop(["Sleep Onset","Sleep Onset","Wake Onset","Last Nap End Time","timezone_offset","Nap Time in Bed","Nap Disturbances","Nap REM","Nap SWS","Nap Light","Nap Wake","Nap Missing Data"],axis=1)
data_RHR.drop(data_RHR.iloc[:, 0:4], inplace = True, axis = 1) 
List_RHR=[2,4,6,7,10,20,35,43,58,64,65,67,80,93,97,101,110,126,137,150,169,174,180,181,190,199,200,210,273,300,301,302,35,380,392,400,427,486,500,572,692,743,785,854,898,910,1011,1058,1090,1100,1111,1112,1156,1179,1290,1365,1389,1399,1400,1401,1402,1450,1468, 1478,1488, 1495,1590,1580,1570,1600,1624,1627,1678,1682,1699,1700, 1710,1743, 1738, 1768,1794,1800, 1805]
for i in List_RHR:
  data_RHR["RHR"][i]=pd.np.nan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [22]:
missing_columns=["RHR","Wake Periods","Sleep Consistency","Respiratory Rate", "Total Cycle Sleep Time (hours)","REM Percentage", "Deep Sleep Percentage", "Restorative Sleep (hours)", "Restorative Sleep (%)"]


In [23]:
def random_imputation(df, feature):

    number_missing = df[feature].isnull().sum()
    observed_values = df.loc[df[feature].notnull(), feature]
    df.loc[df[feature].isnull(), feature + '_imp'] = np.random.choice(observed_values, number_missing, replace = True)
    
    return df

In [24]:
for feature in missing_columns:
    data_RHR[feature + '_imp'] = data_RHR[feature]
    data_RHR = random_imputation(data_RHR, feature)

In [38]:
deter_data_RHR = pd.DataFrame(columns = ["Det" + name for name in missing_columns])

for feature in missing_columns:
        
    deter_data_RHR["Det" + feature] = data_RHR[feature + "_imp"]
    A=data_RHR.copy()
    parameters = list(set(data_RHR.columns) - set(missing_columns) - {feature + '_imp'})
 
    imputer = SimpleImputer(missing_values=np.nan,strategy="mean")
    imputer.fit(A.iloc[:,0:])
    A.iloc[:,0:] = imputer.transform(A.iloc[:,0:])
    
    #Create a Linear Regression model to estimate the missing data
    model = linear_model.LinearRegression()
    model.fit(X = A[parameters], y = data_RHR[feature + '_imp'])
    
    #observe that I preserve the index of the missing data from the original dataframe
    deter_data_RHR.loc[data_RHR[feature].isnull(), "Det" + feature] = model.predict(A[parameters])[data_RHR[feature].isnull()]
    
    #predict=model.predict(A[parameters])
    #std_error = (predict[data[feature].notnull()] - data.loc[data[feature].notnull(), feature + '_imp']).std()
    
    #print(std_error)

print(deter_data_RHR)
data_RHR_O = pd.read_csv("Whoop-Data.csv")
C=pd.concat([data_RHR_O["RHR"],deter_data_RHR["DetRHR"]], axis=1)
C["Raw Bias"]=C["RHR"]-C["DetRHR"]
C["% Bias"]=(C["Raw Bias"]/C["DetRHR"])*100
C.to_csv('/content/drive/MyDrive/SDA/Dataset/LR_RHR_evaluation.csv')

         DetRHR  ...  DetRestorative Sleep (%)
0     51.000000  ...                      43.3
1     54.000000  ...                      48.3
2     58.902737  ...                      43.7
3     49.000000  ...                      54.8
4     63.905371  ...                      48.0
...         ...  ...                       ...
1805  60.175307  ...                      53.8
1806  57.000000  ...                      53.8
1807  56.000000  ...                      58.8
1808  52.000000  ...                      54.0
1809  64.000000  ...                      46.8

[1810 rows x 9 columns]


In [39]:
random_data_RHR = pd.DataFrame(columns = ["Ran" + name for name in missing_columns])
for feature in missing_columns:
        
    random_data_RHR["Ran" + feature] = data_RHR[feature + "_imp"]
    B=data_RHR.copy()
    parameters = list(set(data_RHR.columns) - set(missing_columns) - {feature + '_imp'})
 
    imputer = SimpleImputer(missing_values=np.nan,strategy="mean")
    imputer.fit(B.iloc[:,0:])
    B.iloc[:,0:] = imputer.transform(B.iloc[:,0:])
    
#Create a Linear Regression model to estimate the missing data
    model = linear_model.LinearRegression()
    model.fit(X = B[parameters], y = data_RHR[feature + '_imp'])
    
    predict=model.predict(B[parameters])
    std_error = (predict[B[feature].notnull()] - B.loc[B[feature].notnull(), feature + '_imp']).std()
    random_predict = np.random.normal(size = A[feature].shape[0], loc = predict, scale = std_error)
    random_data_RHR.loc[(A[feature].isnull()) & (random_predict > 0), "Ran" + feature] = random_predict[(A[feature].isnull()) & (random_predict > 0)]

random_data_RHR
#Z=pd.DataFrame(deter_data)
#Z.to_csv('/content/drive/MyDrive/SDA/Dataset/SLR_modified_columns_final.csv')
data_RHR_O = pd.read_csv("Whoop-Data.csv")
D=pd.concat([data_RHR_O["RHR"],random_data_RHR["RanRHR"]], axis=1)
D["Raw Bias"]=D["RHR"]-D["RanRHR"]
D["% Bias"]=(D["Raw Bias"]/D["RanRHR"])*100
D.to_csv('/content/drive/MyDrive/SDA/Dataset/SLR_RHR_evaluation.csv')